In [1]:
import tensorflow as tf

In [2]:
# A tensor is a multidimensional array
# Tensorflow provides a rich library of operations

print(tf.add(5,6))
print(tf.add([1,2],[3,4]))
print(tf.square(7))
print(tf.reduce_sum([1,2,3,4]))

tf.Tensor(11, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(49, shape=(), dtype=int32)
tf.Tensor(10, shape=(), dtype=int32)


In [3]:
# Each tf.Tensor has a shape and a datatype
x = tf.matmul([[1]], [[2, 3]])

print(x)
print(x.shape)
print(x.dtype)
print(x.device)  # Gives the name of the device where the Tensor is hosted

tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>
/job:localhost/replica:0/task:0/device:CPU:0


These tensors can be used be acceleratos (such as GPUs and TPUs) and are immutable
Tensorflow operations automatically convert NumPy array to tensors and vice versa

Tensors are explicitly converted to NumPy ndarrays using their .numpy() method. These conversions are typically 
cheap since the array and tf.Tensor share the underlying memory representation, if possible. However, sharing the 
underlying representation isn't always possible since the tf.Tensor may be hosted in GPU memory while NumPy arrays 
are always backed by host memory, and the conversion involves a copy from GPU to host memory.


Refer to https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/customization/basics.ipynb for usage of devices(GPUs, CPUs, TPUs) for execution, creating datasets from your data and applying transformations to it.



In [4]:
# Keras API provides a way to define composition of computation layers (which are treated as objects)
layer = tf.keras.layers.Dense(10)
# Here the argument denotes the dimension of the output. The dimensions of the 
# can be inferred the first time the code is run or it can be specified
# if the model is complex (or for better readability)

layer = tf.keras.layers.Dense(10, input_shape=(None,5))

# Here the None means that any dimension can be present

In [5]:
# Simply call a layer to use it
layer(tf.zeros([4,5]))

<tf.Tensor: shape=(4, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [6]:
# Inspect the trainable weights and biases in a layer
layer.variables  # or layer.trainable_variables

# Also see layer.kernel and layer.bias

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.00883865, -0.50983137,  0.37399775, -0.3899641 , -0.30502924,
         -0.42187548,  0.10850126, -0.03228277, -0.3271194 , -0.30294174],
        [ 0.33899587, -0.00369632,  0.04724449,  0.61663824,  0.19189924,
          0.32556325, -0.4162503 , -0.29225788, -0.57006425, -0.4859227 ],
        [-0.17266852, -0.10557002, -0.41254842, -0.560386  , -0.3299912 ,
         -0.5113966 ,  0.08524972, -0.373415  , -0.2149665 ,  0.6065716 ],
        [ 0.31679893, -0.14240181,  0.29828793, -0.04239887, -0.32909355,
          0.50690156, -0.5499274 , -0.3822565 , -0.33659095, -0.04671735],
        [ 0.5310922 , -0.55598325, -0.2470896 ,  0.26982856,  0.20577502,
         -0.5185497 , -0.19254524, -0.254123  ,  0.59929687, -0.15297228]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

# Create a model and visualize it

In [7]:
# Import keras and check Tf version
from tensorflow import keras
from tensorflow.keras import layers

tf.__version__

'2.1.0'

In [8]:
# Create a model with three layers and visualize it
inputs = keras.Input(shape=(784,))  # An input layer with 784 features (for MNIST)
dense = layers.Dense(64, activation="relu")
x = dense(inputs) # This is like drawing an arrow from the input to the layer that we created
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(10)(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="mnist_model")  # Here we compile our lauers into a model
model.summary()

Model: "mnist_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Can also plot the graph for the model
# keras.utils.plot_model(model, "my_model")